# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(sorted(file_path_list))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    #c = 0
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #c += 1
            #print(line)
            full_data_rows_list.append(line) 
    
    # to check number of rows in each csv file
    #print(c) 
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## Checking if sessionID is a unique across users of the app

The following tests are performed to check if the selection of primary keys for the three tables below are valid.
First we want to check if sessionID is unique for each session and does not overlap across users.

In [9]:
df = pd.read_csv("event_datafile_new.csv")
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [11]:
# To check if sessionId is unique or if there could be overlap between users

# First create a dataframe of just sessionId, userID, and counts 
# If the number of unique items in this table is equal to the number of rows, then sessionID is unique
a = df.groupby(['sessionId', 'userId'])['song'].count()
b = pd.DataFrame(a).reset_index()

# Number of session IDs should be equal 
print('The hypothesis that sessionId is unique and not shared by users is:', len(b['sessionId'].value_counts()) == len(b))

The hypothesis that sessionId is unique and not shared by users is: True


Since sessionID is unique we can use sessionID and itemInSession as composite PRIMARY KEYS.

### Create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
The table 'all_sessions' created below contain all the sessions IDs and items played during each session.
The composite primary key is valid because each sessionID is unique and not shared between users.
sessionId is the partition key while itemInSession is the clustering column.

In [12]:
# Create table in Cassandra for Query 1

query = "CREATE TABLE IF NOT EXISTS all_sessions"
query = query + """
                  (sessionId int, 
                   itemInSession int, 
                   artist text, 
                   song text, 
                   length float, 
                   PRIMARY KEY (sessionId, itemInSession))
                """
session.execute(query)

In [13]:
# Load data from the csv file into the Cassandra database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO all_sessions (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

Verify that the data have been inserted into each table:

In [14]:
## Verify the data was entered into the table
query = "SELECT * FROM all_sessions WHERE sessionID = 1049;"
rows = session.execute(query)

# Create a dataframe and display query result
container = [[row.artist, row.song, row.length] for row in rows]
query_result = pd.DataFrame(container, columns = ['artist', 'song', 'length']) 
query_result.head()

,artist,song,length
0,Manowar,Shell Shock,247.561996
1,Morcheeba,Women Lose Weight (Feat: Slick Rick),257.410156
2,Maroon 5,Won't Go Home Without You,231.235458
3,Train,Hey_ Soul Sister,216.763626
4,LMFAO,I'm In Miami Bitch,227.996277


In [15]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song, length FROM all_sessions WHERE sessionId = 338 AND itemInSession = 4;"
rows = session.execute(query1)

# Create a dataframe and display query result
container = [[row.artist, row.song, row.length] for row in rows]
query1_result = pd.DataFrame(container, columns = ['artist', 'song', 'length']) 
display(query1_result)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


The fourth song that was listened to during session 338 is shown above along with the song artist and song duration in seconds. 

## Query 2
 The table 'user_specific_sessions' created below contain all the sessions IDs and songs played for each user. 
 The schema allows for queries on songs played by users. 
 The composite primary key is a combination of userId, sessionId, itemInSession. This guarantees a unique key for each record.
 userId and sessionId are the partition keys in order to efficiently break the data into buckets when retrieving from a large dataset while itemInSession is the clustering column.


In [16]:
# Create table for Query 2
query = "CREATE TABLE IF NOT EXISTS user_specific_sessions"
query = query + """
                  (userId int,
                   sessionId int, 
                   itemInSession int, 
                   artist text, 
                   song text,
                   firstName text,
                   lastName text,
                   PRIMARY KEY ((userId, sessionId), itemInSession))
                """
session.execute(query)

In [17]:
# Load data from the csv file into the Cassandra database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_specific_sessions (userId, sessionId, itemInSession, \
                                                     artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

Verify if data was entered into the table:

In [18]:
## Verify if data was entered into the table
query = "SELECT * FROM user_specific_sessions WHERE userId = 36 AND sessionId = 998;"
rows = session.execute(query)

# Create a dataframe and display query result
container = [[row.artist, row.song, row.firstname, row.lastname] for row in rows]
query_result = pd.DataFrame(container, columns = ['artist', 'song', 'firstName', 'lastName']) 
query_result.head()

,artist,song,firstName,lastName
0,Alex Ubago,Sin Miedo a Nada (con Amaia Montero),Matthew,Jones
1,The Twilight Singers,Bonnie Brae,Matthew,Jones
2,Chikita Violenta,War,Matthew,Jones
3,Cat Stevens,Sad Lisa,Matthew,Jones
4,Kenny Garrett,The House That Nat Built (Album Version),Matthew,Jones


In [19]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = "SELECT artist, song, firstName, lastName FROM user_specific_sessions WHERE userId = 10 AND sessionId = 182;"
rows = session.execute(query2)

# Create a dataframe and display query result
container = [[row.artist, row.song, row.firstname, row.lastname] for row in rows]
query2_result = pd.DataFrame(container, columns = ['artist', 'song', 'firstName', 'lastName']) 
display(query2_result)

,artist,song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


The name of songs played by userId 10 during sessionId 182 are shown above along with the artist and user name. The songs are ordered by iteminSession in ascending order since iteminSession is one of the clustering key in the primary key. 

## Query 3

The table 'songs_by_user' contains all the songs played by a particular user.
The composite primary key is a combination of song title, and userId.
The partition key is the song title whilee the userId serves as the clustering key.

In [21]:
# Create table in Cassandra for Query 3

query = "CREATE TABLE IF NOT EXISTS songs_by_user"
query = query + """
                  (song text,
                   userId int,
                   firstName text, 
                   lastName text,
                   PRIMARY KEY (song, userId))
                """
session.execute(query)

In [22]:
# Load data from the csv file into the Cassandra database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

Verify if data was entered into the table:

In [23]:
## Verify if data was entered into the table
query = "SELECT * FROM songs_by_user WHERE song = 'Playboy'"
rows = session.execute(query)

# Create a dataframe and display query result
container = [[row.firstname, row.lastname] for row in rows]
query_result = pd.DataFrame(container, columns = ['firstName', 'lastName']) 
query_result.head()

,firstName,lastName
0,Jacob,Klein


In [25]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = "SELECT firstName, lastName FROM songs_by_user WHERE song = 'All Hands Against His Own';"
rows = session.execute(query3)

# Create a dataframe and display query result
container = [[row.firstname, row.lastname] for row in rows]
query3_result = pd.DataFrame(container, columns = ['firstName', 'lastName']) 
display(query3_result)
                    

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


The above table shows the users who have listened to the song "All Hands Against His Own" throughout the history of the Sparkify music app.

### Drop the tables before closing out the sessions

In [26]:
## Drop the table before closing out the sessions

query = "drop table all_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_specific_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_in_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


Error from server: code=2200 [Invalid query] message="unconfigured table songs_in_sessions"


### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()